## QCoDeS Training for Developers

Welcome to the training about QCoDeS targeted for developers. After this training, developers should have more knowledge about QCoDeS which will bring them confidence with contributing to it.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

### Approach

- Don't bring too much relevant background, only small absolutely necessary portions
- Quickly/slightly refresh the user-side view on QCoDeS, and get close to code
- Describe features/concepts from ideas, but get close to code very quickly

### Execution plan for presenter

- Before:
  - Have qcodes code at hand in your IDE https://github.com/qcodes/qcodes
  - Run the training Jupyter notebook, and have it at hand https://github.com/astafan8/Qcodes/blob/training-for-devs/docs/examples/QCoDeS%20Training%20for%20Developers.ipynb
  - Familiarize yourself with the following info from the docs:
    - Front page https://qcodes.github.io/Qcodes/index.html
    - 15 minutes of QCoDeS https://qcodes.github.io/Qcodes/examples/15_minutes_to_QCoDeS.html
    - `Parameter`s
      - overview1 https://qcodes.github.io/Qcodes/examples/Parameters/Parameters.html
      - overview2 https://qcodes.github.io/Qcodes/examples/writing_drivers/Creating-Instrument-Drivers.html#What%E2%80%99s-a-Parameter? - only this section
      - API docs https://qcodes.github.io/Qcodes/api/parameters/parameter.html#module-qcodes.instrument.parameter
      - `ParameterWithSetpoints` https://qcodes.github.io/Qcodes/examples/Parameters/Simple-Example-of-ParameterWithSetpoints.html
    - Performing measurements https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html
    - dataset notebooks
      - the class https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html
      - data access https://qcodes.github.io/Qcodes/examples/DataSet/Accessing-data-in-DataSet.html
- During:
  - Go through notebook in 3 parts, have 5 min breaks, 5 min reserve beginning, 10 min reserve at the end
  - Have a time keeper
  - Let ask questions at specific moments but very frequently to feel interactive
- After:
  - Port notebook to qcodes docs ASAP
  - Sessions on some parts if requested

### Imports

In [2]:
import qcodes

from typing import Optional, Sequence, Dict
from pprint import pprint

import numpy

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-miasta\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-miasta\.qcodes\logs\200415-28860-qcodes.log


## QCoDeS? ah? [5min]

What is it? What are it's goals, and users? And why?

### "The It"

**Q.** uantum **Co** penhagen **De** lft **S** ydney:

- data acquisition framework
- modular
- python-based
- open-source

### Goal

**Common** framework for physics experiments:

- users need to write only their-own-experiment-specific code
- physics experiments can take advantage of modern software and best practices
- code can and should be contributed back to the framework
- the process of moving between teams or labs, and of setting up a new experiment is streamlined
- new students don’t need to spend a long time learning software in order to participate in experiments

### Users

- people doing measurements:
  - academics: e.g. scientists and students
  - corporates: e.g. experimentalists, engineers, hardware manufacturers
- worldwide

### Driven by

- Microsoft, largely
- worldwide comminity, growing

### Development principles

- Robust code - as opposed to what users are used to from other measurement frameworks
- Respect backwards compatibility - don't break user's code that "works"!
- Modularity - extend-existing, not go-off-and-write-your-own
- User facing API - list in docs, devs can use everything as appropriate
- All of the above is basically maintatining users' trust
- Carefully scoping problems

## I wanna MEASURE [30min]

How do I measure things? How are measurements enabled via QCoDeS?

We will be looking into the following pyramid of objects and concepts:
- Station - the ultimate bucket
  - Instruments - and channels
    - Parameters - many kinds

### Station

``Station`` is just a bucket for components such as instruments, parameters, etc.

Speaking more strictly, those components should be snapshot-able (subclasses of ``Metadatable``), but more about this in the section about the snapshot.

``Station`` can be created with python code and/or using YAML configuration file. [Read on Station, its interface, YAML config, and more here](http://qcodes.github.io/Qcodes/examples/Station.html).

In [3]:
from qcodes.station import Station

#### Default station

Notice the ``default=True`` argument of the ``Station`` - it controls wether to store the initialized ``Station`` object into an attribute.

Using ``Station.default`` as a fallback in other objects in QCoDeS is very common, as it is extremely uncommon that more than one ``Station`` is initiated in one python session.

In [4]:
station = Station()  # default=True

assert Station.default is station

station2 = Station(default=False)

assert station2 is not Station.default

### Instruments

Instrument conceptually represents hardware that one needs to talk to in order to perform measurements.

The concept can be expanded to also include "instruments" that are:

- not remotly controllable but it's important to record their state
- exposing a more measurement-specific interface while aggregating actual instruments

#### Drivers

Concrete implementation of classes and abstracations provided by QCoDeS for particular piece of hardware is commonly referred to as ``drivers`` or ``QCoDeS drivers`` for that piece of hardware.

From a real conversation:

> - Shall we buy this useful, powerful, and expensive measurement equipment?
> - Hmm... Is there a QCoDeS driver for it?

#### Base class(es)

From coding perspective, instruments in QCoDeS are subclasses of ``Instrument`` and ``InstrumentBase`` from ``qcodes.instrument.base``.

``InstrumentBase`` - defines interface:

- has a name
- is snapshot-able (subclass of ``Metadatable``)
- serves as a bucket for ``Parameter``s
- may include snapshot-able "submodules"
  - navigation via ``root_instrument``, ``parent``, ``ancestors``
- instance-bound logger
- convenient dot-notation access to it's parameters and submodules (thanks to ``DelegateAttributes``)
- convenient ``[]``-notation access to them as well

``Instrument`` - accounts for communication:

- subclass of ``InstrumentBase``
- hides communication with hardware via ``ask``/``write``
- implements global registry of instrument instances
- ``IDN``, ``connect_message``

#### Submodules

Submodules allow for logical grouping of functionality within an instrument. For example, [look at the trigger, sample, and display submodules of Keysight 344xxA driver](https://qcodes.github.io/Qcodes/examples/driver_examples/Qcodes%20example%20with%20Keysight%20344xxA.html#Parameters-and-methods-overview) which allow to conveniently group parameters and methods without polluting the root namespace of the driver.

To add a submodule to an instrument, use it's ``add_submodule`` method (usually in instrument's ``__init__``).

``InstrumentBase`` promises to include snapshots of it's submodules in its own snapshot.

How to implement submodules may not be clear:
- the current implementation suggests that any ``Metadatable`` object will work
- the typing of ``add_submodule`` suggests using/subclassing ``InstrumentBase`` in order to take full advantage of the ``InstrumentBase``s interface for the submodule
- most commonly used/cubclasses is ``InstrumentChannel`` from ``qcodes.instrument.channel`` because
  - it takes care of correct naming and 
  - forwards communication to the parent instrument

#### Channels

Channels are most popular submodules of instruments. Why? Think of a multi-channel voltage source kind of instrument.

QCoDeS provides ``InstrumentChannel`` class that takes care of correct naming and forwards communication to the parent instrument. ``InstrumentChannel`` can be added as submodules to the instrument.

Moreover, QCoDeS provides ``ChannelList`` class. In short, it allows to group channels that are of the same class, and implements convenient access to common parameters and functions of those channels. See the difference between the following ways of setting voltage parameter of all channels to ``0``:
```python
my_voltage_source.channels.voltage(0)
```
versus
```python
for channel in my_voltage_source.channels:
    channel.voltage(0)
```
or even worse
```python
channel_names = [...]  # I just happen to know them
for channel_name in channel_names:
    channel = my_voltage_source[channel_name]
    channel.voltage(0)
```

#### Hardware communication

``Instrument`` class suggests the most frequent use case of how communication with hardware is being performed:
- ``write(cmd: str) -> None``
- ``ask(cmd: str) -> str``

Subclasses of ``Instrument`` should implement concrete communication by overriding ``write_raw`` and ``ask_raw``.

A lot of QCoDeS abstractions are built with this string-command-based communication model in mind, especially ``Parameter``.

##### ``VisaInstrument``

A great example is ``VisaInstrument`` from ``qcodes.instrument.visa`` - it implements ``write_raw`` and ``ask_raw`` on top of ``pyvisa`` library that provides a python wrapper over implementations of VISA (Virtual Instrument Software Architecture) API.

Here's a schematic of the layering, from high, user-facing, to low:
```
- MyInstrumentDriver . parameter/method
- VisaInstrument     . ask/write
- Instrument         . ask_raw/write_raw
- pyvisa .. Resource . query/write
- Actual hardware communication layer, e.g. LAN, USB
  implemented by a VISA driver (e.g. National Instruments, pyvisa-py, etc)
```

The command API of the most VISA instruments conforms to SCPI (Standard Commands for Programmable Instruments) standard which makes QCoDeS driver development for those instruments extremely easy.

##### Other

QCoDeS does **not** restrict interacting with hardware only with this communication model. An good example of this are instruments which use some specific ``.dll`` library, see drivers for [Alazar high-speed digital acquisition cards](https://github.com/QCoDeS/Qcodes/blob/master/qcodes/instrument_drivers/AlazarTech/ATS.py#L68), [Vaunix step attenuator](https://github.com/QCoDeS/Qcodes_contrib_drivers/blob/c007532d48dfc035a9e2f03167606c16c42d7f77/qcodes_contrib_drivers/drivers/Vaunix/LDA.py#L59), etc.

In that case, ``write_raw``/``ask_raw`` are usually left as raising ``NotImplementedError``, and parameters and methods of the instrument driver don't use them at all.

#### Registry of instances

QCoDeS does not allow creation of an instrument with the same ``name``. Why? - To enforce that there is **only one** connection to an instrument.

##### No instruments with same name

Let's try to create two instruments with the same name:

In [5]:
from qcodes.instrument.base import Instrument

foo_called_foo = Instrument('foo')

In [6]:
try:
    bar_called_foo = Instrument('foo')
except:
    import traceback
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-6-1e60ed53d4d7>", line 2, in <module>
    bar_called_foo = Instrument('foo')
  File "c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument\base.py", line 431, in __init__
    self.record_instance(self)
  File "c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument\base.py", line 567, in record_instance
    raise KeyError('Another instrument has the name: {}'.format(name))
KeyError: 'Another instrument has the name: foo'


In order to get rid of the exception, the user needs to ``close`` one of the instruments:

In [7]:
foo_called_foo.close()

In [8]:
bar_called_foo = Instrument('foo')
bar_called_foo

<Instrument: foo>

##### Fool proof?

This may not be the optimal mechanism.

For example, one can fool the mechanism by using two different names but the same VISA address:
```python
foo_at_address1 = VisaInstrument('foo', address='address1')
bar_at_address1 = VisaInstrument('bar', address='address1')
# DISCLAIMER: only tried with pyvisa-sim, not a real VISA driver
```
in which case it's now up to the "connectivity backend" (in this case, VISA) to deal with having multiple "connections" to the same piece of hardware.

##### Behind the scenes

Behind the scenes QCoDeS maintains a registry of instrument instances by their names. The scope of the registry is the python process.

``Instrument`` class maintains an ``_all_instruments`` name-to-weak-reference dictionary in a class attribute.

Each ``Instrument`` subclass maintains its own ``_instances`` list of weak references to instances of **only that (sub)class** as in a class attribute (and as ``instances`` method).

``Instrument`` class provides a few class methods for navigating in the registry: ``instances``, ``find_instrument``, ``exists``, ``is_valid``. The most popular however are ``close_all`` method and ``find_or_create-instrument`` function.

##### ``find_or_create_instrument``

There is a ``find_or_create_instrument`` convenience function that allows to conveniently find an existing instrument, create it if it doesn't exists, or re-create it if it does exist:

In [9]:
from qcodes.instrument.base import find_or_create_instrument

In [10]:
# Example of Find
foo = find_or_create_instrument(Instrument, 'foo')
assert foo is bar_called_foo

Connected to: None foo (serial:None, firmware:None) in 0.08s


In [11]:
# Example of Recreate
foo = find_or_create_instrument(Instrument, 'foo', recreate=True)
assert foo is not bar_called_foo

In [12]:
# Example of Create
class MyInstrument(Instrument):
    pass

foo = find_or_create_instrument(MyInstrument, 'bar')

##### ``close_all``

Another useful API is ``close_all`` - it closes all instrument instances.

In [13]:
print('before closing all:', Instrument._all_instruments)

Instrument.close_all()

print('after closing all:', Instrument._all_instruments)

before closing all: {'foo': <weakref at 0x0000027AF08324A8; to 'Instrument' at 0x0000027AF228CCC8>, 'bar': <weakref at 0x0000027AF2275F98; to 'MyInstrument' at 0x0000027AF227A0C8>}
after closing all: {}


##### And ``Station``?

``Station`` is aware of the instrument instance registry. It exposes convenient API which is station-scoped version of the above-mentioned ``Instrument`` APIs:

- ``.load_instrument(.., revive_instance=False, ..)`` - for ``find_or_create_instrument``
- ``.close_and_remove_instrument(..)`` - for ``Instrument.close``
- ``.close_all_registered_instruments(..)`` - for ``Instrument.close_all``

### Parameters

Parameters in QCoDeS serve mostly two purposes:

- values that can be **set** or **measured** via an instrument, e.g. voltage, current, time trace
- "settings" of an instrument, e.g. range, precision, NPLCs

**NOTE** that context of an experiment/measurement is important. The same parameter may serve either of the purposes depending on the context. Think of an offset of a sine wave - in one experiment that will be just a setting, in another it will "set" to different values (e.g. swept over).

In QCoDeS, parameter is modeled as an object that is:

- representing a value (in broad sense)
- and metadata about that value,
- and that value can, or can't, be ``set`` and ``get``

#### Base class(es)

The main base class for parameters in QCoDeS is ``Parameter`` from ``qcodes.instrument.parameter``.

All parameters should be created/instantiated from ``Parameter``, and more sophisticated parameters should be implemented as subclasses ``Parameter``.

However, ``Parameter`` inherits from a ``_BaseParameter``. The separation between the two may not be clear from the code, yet let's try to see it below:

``_BaseParameter`` - the "meat":

- has a name
- implements ``set``/``get`` methods
  - via wrapping ``get_raw``/``set_raw`` that are to be overridden
  - ``__call__`` convenience
- built-in ``set``/``get`` behavior controls:
  - value validators
  - ``get_parser``, ``set_parser``
  - ``val_mapping``
  - ``scale``, ``offset``
  - ``step``
  - ``inter_delay``, ``post_delay``
- has ``cache`` of the parameter value
  - and, ``get_latest``, cache's predecessor
- may be aware of the instrument it belongs to
- is snapshot-able (subclass of ``Metadatable``)
  - implements flags that control how snapshot is being taken

``Parameter`` - the important extension:

- subclass of ``_BaseParameter``
- adds ``label``, ``unit``, ``docstring``
- enables strings, callables, ``None`` and ``False`` as ``set``/``get``
- interfaces for the legacy ``Loop`` feature

#### Name, label, unit, ...

Metadata is a very important part of the parameter. It allows to reason about the meaning of its value, and ``set``/``get`` commands (if any). Hence, it is important to assign good names, labels, units, and a docstring.

``name`` is NOT supposed to be changed after parameter's initialization.

``label`` and ``unit`` are plain attributes of ``Parameter`` and are not recommended to be changed after initialization.

However, it is tempting to adjust ``label``/``unit`` sometimes. When a parameter represents some sort of an acquisition buffer of an instrument which can hold both, for example, voltage and current values, it'd be great to change ``label``/``unit`` to ``"Voltage"``/``"V"`` when the instrument is aquiring voltage, and to ``"Current"``/``"A"`` when it's aquiring current. Note that unfortunately, the docstring of the parameter (the ``__doc__``) will still use ``label``/``unit`` that the parameter was initiated with. That is to say that not all ``Parameter`` features are implemented with change-ability of ``label``/``unit`` in mind.

Below a few parameters are created, and their final docstring is printed.

In [14]:
from qcodes.instrument.parameter import Parameter

delay = Parameter(
    'delay',
    label='Trigger Delay',
    unit='s',
    docstring='Delay between the trigger signal and the first measurement'
)

voltage = Parameter(
    'voltage',
    label='Voltage',
    unit='V',
    docstring='Measured voltage'
)

In [15]:
print(delay.__doc__)

Delay between the trigger signal and the first measurement

Parameter class:

* `name` delay
* `label` Trigger Delay
* `unit` s
* `vals` None


In [16]:
print(voltage.__doc__)

Measured voltage

Parameter class:

* `name` voltage
* `label` Voltage
* `unit` V
* `vals` None


#### ``add_parameter`` of ``Instrument``

Parameters are usually conveniently grouped into instruments or their submodules. This is why ``InstrumentBase`` implements a convenient ``add_parameter`` method - it's purpose is to:

- initiate a parameter of optionally provided class with provided arguments,
- ensure that the instrument instance is passed to the parameter, and
- add the initiated parameter to instruments collection of parameters (in ``.parameters``).

It is important that ``add_parameter`` ensures to pass to the instrument instance to the parameter. Thanks to this:
- the parameter can use the instrument instance via its ``.instrument`` and ``root_instrument`` properties
- parameter can leverage the instrument's ``write``/``ask`` methods for communication with the hardware
- parameter's ``full_name`` gets prefixed by the instrument's name (or, more broadly, by the channel's or submodule's name)

#### ``set`` and ``get``

``set`` and ``get`` methods of a parameter are it's primary interface for setting and getting it's value:

* ``.set(value: ParamDataType) -> None``
* ``.get() -> ParamDataType``

(``ParamDataType`` is an alias for ``Any``, and is used mostly for readability.)

Actual signatures are more flexible - ``kwargs`` for both and even ``args`` for ``get``:
* ``.set(value: ParamDataType, **kwargs: Any) -> None``
* ``.get(*args: Any, **kwargs: Any) -> ParamDataType``

Usage of ``args`` and ``kwargs`` is discouraged in order not to pollute the simplicity of the ``Parameter``'s interface.

Where does the ``value`` come from or get sent to? It can be:

- some instrument - communication with hardware
- memory - value stored in memory, and is just useful in a particular context
- **anything** - ``set``/``get`` are just callables, they can really do anything about the value :)

``set`` and ``get`` methods are not explicit bound methods of a parameter. They are created during ``__init__`` by wrapping other methods which concretely implement what needs to happen when a value is being ``set`` or ``get``.

The ways to provide these concrete implementations for wrapping into actual ``set``/``get`` methods are described below.

##### ``set_raw`` and ``get_raw``

The primary way of implementing behavior of ``set``/``get`` is to override ``set_raw``/``get_raw`` methods of ``Parameter`` (well, actually ``_BaseParameter``). Behind the scenes, calling ``set``/``get`` will forward the call to ``set_raw``/``get_raw``.

Below we implement a trivial parameter that accesses a variable in outer scope. (It is convenient to use for demonstration, but never do this in production :) )

In [17]:
precious_value = 42

class PreciousParameter(Parameter):
    def set_raw(self, value):
        global precious_value
        precious_value = value

    def get_raw(self):
        global precious_value
        return precious_value

In [18]:
my_precious = PreciousParameter(
    'my_precious',
    label='My Precious Value',
    unit=''  # precious things are priceless, unitless
)

# Value gotten via the parameter should equal to ``precious_value``
parameter_value = my_precious.get()
assert parameter_value == precious_value

# Setting parameter value should be reflected in ``precious_value``
my_precious.set(66)
assert precious_value == 66

# Getting parameter value again should still return the new ``precious_value``
parameter_value = my_precious.get()
assert parameter_value == precious_value

##### ``set_cmd`` and ``get_cmd``

Historically, more flexible and convenient ways of defining parameter's ``set``/``get`` methods behavior were needed. This is how ``Parameter`` (not ``_BaseParameter``) got two interesting arguments - ``set_cmd`` and ``get_cmd``. Let's go through the "values" that can be supplied with these arguments.

###### ``False`` - non-get-able/non-set-able parameter

If a ``Parameter`` is initialized with the set/get commands set to ``False`` then its not settable/gettable. This means that the instance will not have the ``set``/``get`` methods respectively.

* Having a set-only or a get-only parameter may be useful
  * for example for measurement parameters, or settings/characteristics that an instrument calculates internally
* If a parameter is set-only or get-only, it is still possible to access its cache which is useful when implementinga driver.
* It is rarely useful to have a parameter that can't be both set and gotten.
* ``set_cmd=False`` is ``Parameter``s default, likely due to frequent occurence of get-only instrument parameters.

###### ``None`` - "memory parameter", store/take value in/from memory

If a ``Parameter`` is initialized with commands set to ``None``, then parameter's internal cache will be the only place where the value is stored.

* It is possible to pass only one of the ``set_cmd``/``get_cmd`` as ``None``
  * The reason this works is that parameter's cache is used anyway to store the value,
    so any additional behavior of ``set``/``get`` is an "extra".
  * This can be useful - for example, if an instrument permits to only get or only set some settings,
    it may be convenient to rely on parameter's cache for the not-permitted operation.
* ``get_cmd=None`` is ``Parameter``s default, likely due to frequent occurence of set-only instrument parameters.

###### String ``Command``s - convenient for SCPI (all text-based) instruments

Text-based communication is very popular among instruments. Most of them even conform to the SCPI standard of the text-based instrument API.

Now, let's at what pattern is frequently arises because of this - let's try to implement a driver for a VISA instrument that uses text-based communication with one parameter.

In [3]:
from qcodes.instrument.visa import VisaInstrument
from qcodes.instrument.parameter import Parameter


class Meter(VisaInstrument):
    # This will be a simple instrument that can measure
    # 'a feeling of temperature' and return a string.
    # It also allows us to specify a range which increases
    # its vocabulary of temperature feelings.

    def __init__(self, name, address):
        # skipping many other arguments for simplicity
        super().__init__(name, address)
        
        self.add_parameter('measure', parameter_class=MeasureParameter)
        self.add_parameter('range', parameter_class=RangeParameter)


class MeasureParameter(Parameter):
    # In order not to make this example too complex,
    # let's assume that the instrument returns a string
    # value, say, "HOT"/"WARM"/"COOL"/"COLD"
    def get_raw(self):
        return self.instrument.ask('MEASURE?')


class RangeParameter(Parameter):
    # In order not to make this example too complex,
    # let's assume that the instrument specifies the range
    # via a string value, say, "OUTDOORS"/"INDOORS"

    def set_raw(self, value):
        return self.instrument.write(f'RANGE {value}')

    def get_raw(self):

        return self.instrument.ask('RANGE?')

Notice that both parameters use instrument's ``ask``/``write`` methods. Moreover, both of the parameter classes just implement usage of different string commands, they do nothing more.

For such a simple instrument, this is not that bad, yet still quite some boilerplate in order to create just two parameters with the only purpose of providing a command string. Imagine a more realistic driver - that one would usually have 10s of parameter, hence more boilerplate code with this approach.

Let's compare the above code with the following code that uses ``set_cmd``/``get_cmd``:

In [4]:
class SmartMeter(VisaInstrument):
    # Same as ``Meter`` but fewer lines of code

    def __init__(self, name, address):
        super().__init__(name, address)

        self.add_parameter('measure', get_cmd='MEASURE?')
        self.add_parameter('range', set_cmd='RANGE {}', get_cmd='RANGE?')

This is the same functionality but in much more concise code.

So, how do ``set_cmd``/``get_cmd`` work? ``Command`` object from ``qcodes.utils.command`` encapsulates the functionality.

Here is how ``set_cmd``/``get_cmd`` are processed:

- ``Parameter`` notices that ``set_cmd``/``get_cmd`` is not a ``None`` or ``False``
- ``Parameter`` creates a ``Command`` object
- the value of ``set_cmd``/``get_cmd`` is passed to ``Command``
- parameter's instrument's ``write``/``ask`` methods are also passed to ``Command``
- The created ``Command`` object is callable, hence can be assigned as ``set``/``get`` method of the ``Parameter``
- ``Command`` treats the passed ``set_cmd``/``get_cmd`` string as a python format string (thus enabling value-setting kind of commands)
- When called ``Command`` formats the command string with given value, and the formatted string is passed to the previously-given ``write``/``ask`` instrument methods.

String-based no-value and single-value commands are very common among instruments, and thanks to ``set_cmd``/``get_cmd`` can be easily implemented.

Note that historically ``Command`` class itself is very flexible, and not all of it's features are used.

###### Callable ``Command``s

Sometimes string commands are not enough to implement the behavior of parameter's ``set``/``get`` methods. At the same time, creating a separate subclass of ``Parameter`` and implementing its ``set_raw``/``get_raw`` seems an overkill.

Solution to that is to pass callables to ``set_cmd``/``get_cmd`` (functions or methods). This is one of the *useful* flexibilities of the ``set_cmd``/``get_cmd`` and ``Command`` class.

One of the most frequent use cases for callable ``set_cmd``/``get_cmd`` is when setting a parameter induces changes in other parameters on the instrument such we need to request those values explicitly in order to make sure that the driver correctly represents the state of the instrument.

#### ``set_to``

An important convenience that parameters implement is a ``set_to`` context manage that allows to set the parameter value to a given value within the context, and then revert its value to the one that the parameter had before the context.

This feature is important because it removes the need for users remember to revert the parameter to its original value (if relevant). At a larger scale, this feature provides convenience for putting an instrument in a desired state and automatically reverting from it, in case the state is represented by a *number* of parameters.

The usage of the context manager looks like this:
```python
param.set(original_value)
with param.set_to(new_value):
    print(param())
    # here `param`'s value is `new_value`
print(param())
# here `param`'s value is again `original_value`
```

This is possible because parameter maintains a cache of it's value, hence it is aware of its "original" value, and hence the content manager can revert the parameter to it.

If a parameter has never been set or gotten before using the ``set_to`` context, if possible, it's value will be requested by calling parameter's ``get``. This is thanks to the smartness of the parameter's cache.

#### ``cache`` and ``get_latest``

- interface - it's similar to parameter itself
- get_latest
- what if the parameter have never been gotten
- max_val_age

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

#### Value and raw value

- anatomy of set/get - show sequence
- the from/to parts - show sequence
- Parsers
- Val_mapping
- Validators

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

#### Snapshot

Snapshot - it's nontriviallity due to instrument communication and flags

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

#### More sophistication

just in general about more tough values, and group parameter, and other weird stuff that needs a simple "get".

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

#### ``DelegateParameter``

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

#### Array valued parameters

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

### Snapshot

Snapshot is a "photograph" of a state of a measurement setup or its piece, be it ``Station``, ``Instrument``, ``Parameter``, or other components.

Snapshot is **not** maintained in memory. Instead, each snapshot-able component can always return/produce its snapshot.

#### Subclass ``Metadatable``

Snapshot-able objects should inherit from ``qcodes.utils.metadata.Metadatable`` and should override ``snapshot_base`` method to return a dictionary representing the state of the object. Then, calling ``snapshot()`` method will return the snapshot of the object.

In [18]:
from qcodes.utils.metadata import Metadatable


class TeamsStatus(Metadatable):
    def __init__(self, name: str, status: str, metadata=None):
        super().__init__(metadata=metadata)
        self.name = name
        self.status = status

    def snapshot_base(
            self, update: bool = False,
            params_to_skip_update: Optional[Sequence[str]] = None) -> Dict:
        return {'status': self.status}


teams_status = TeamsStatus('Fridge25 status', 'busy')

pprint(teams_status.snapshot())

{'status': 'busy'}


####  ``metadata`` argument

Note that ``Metadatable`` also implements ``metadata`` keyword argument - ``metadata`` is expected to be a dictionary, and it will be included in the snapshot automatically (if it is not ``{}``).

In [19]:
favorites = Metadatable(metadata={'number': 6 + 9j})
favorites.load_metadata({'fruit': 'manadrine'})  # ``dict.update`` behavior

In [20]:
pprint(favorites.snapshot())

{'metadata': {'fruit': 'manadrine', 'number': (6+9j)}}


#### Add  ``Metadatable`` to station

``Metadatable`` objects can be added to the ``Station``, other objects - should not (otherwise ``snapshot()`` will raise).

In [21]:
station = Station(teams_status)
station.add_component(favorites)

'component1'

In [22]:
pprint(station.snapshot())

{'components': {'Fridge25 status': {'status': 'busy'},
                'component1': {'metadata': {'fruit': 'manadrine',
                                            'number': (6+9j)}}},
 'config': None,
 'default_measurement': [],
 'instruments': {},
 'parameters': {}}


#### Create ``JSON`` snapshot

Although snapshot is a python dictionary, it is always meant to be converted to JSON and stored next to the measured data. QCoDeS maintains it's own JSON converter class, ``qcodes.utils.helpers.NumpyJSONEncoder``, to support conversion of objects that are frequently met in measurement world, such as ``numpy`` arrays, complex numbers, pickle-ables.

In [23]:
from qcodes.utils.helpers import NumpyJSONEncoder
import json

json_snapshot = json.dumps(
    station.snapshot(), cls=NumpyJSONEncoder,
    indent=2, sort_keys=True  # for pretty-printing
)

In [24]:
print(json_snapshot)

{
  "components": {
    "Fridge25 status": {
      "status": "busy"
    },
    "component1": {
      "metadata": {
        "fruit": "manadrine",
        "number": {
          "__dtype__": "complex",
          "im": 9.0,
          "re": 6.0
        }
      }
    }
  },
  "config": null,
  "default_measurement": [],
  "instruments": {},
  "parameters": {}
}


#### ``update`` arguments

Note that ``snapshot`` has an ``update`` kwarg, and ``snapshot_base`` has ``update`` and ``params_to_skip_update`` kwargs - those allow to optimize querying the state of a ``Metadatable`` object.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

## BREAK [5min]

## I wanna STORE [30min]

How and where do I store what I measured? And access back? Database, experiments, datasets, parameters?

- Quickly DB structure, paramtypes, data saving approach
- mention DataSet is too close to sqlite
- dig deeper into how Measurement/DataSet are designed

### Measurement code requirements

Historically data aquisition frameworks have been struggling to provide balance between flexibility and convenience.

- Flexibility of the framework should allow experimentalists to not limit their progress when they have a measurement in mind.
- Convenience of the framework should allow measurement ideas to become measurement code in a short time.

Both these requirements are extremely hard to fulfill. QCoDeS has a legacy ``Loop`` feature that was a great convenience and was flexible enough for quite many straightforward measurements, but would becomes difficult and hacky for other measurements.

At the moment, QCoDeS is priorotizing "flexibility" by taking the following approach - allow freeform python as measurement code but providing clear API for data storage.

### Storing what is measured

Instruments and parameters provide great abstractions to perform measurements: ``set`` this, ``get`` that - great. The question is now how to store the measured data.

What?

- data, the values themselves
- metadata about the values - label, unit, etc
  - structure of the measured data, if possible
- station snapshot
- mesaurement meta information

How?

- persiting data ASAP
- flexible, allowing basically any structure of measured data
- data and metadata conveniently linked
- efficiently, compared to characteristic measurement times
- not allow overwriting to prevent mistakes

This is what ``Measurement``, ``DataSaver`` and ``DataSet`` are intending to implement.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)


	• Measurement -> DataSet
		○ ASSUME THAT 'how to use them' IS KNOWN
		○ Name, experiment, station
		○ Parameters
			§ Register_* - as interface between qcodes parameters and dataset parameters
			§ Rundescription/interdeps/paramspecbase
			§ paramtype
		○ Before and after actions :)
		○ .run() -> Runner -> DataSaver - what is their role?
			§ Runner
			§ DataSaver
				□ Add_results
					® Validation that happens
				□ Dataset/run_id
		○ Sqlite data saving optimizations
			§ Paramtype "array"
			§ wrtie_period
			§ write_in_background
		○ Subscribers - bonus topic
	• Database stuff
		○ Runs table as log of performed measurements
		○ Experiment as a bucket of datasets but not used as such really
			§ Ideas for "active experiment, etc"
		○ Data storage
			§ As a message queue/log - needs reconstruction
			§ in the same DB
		○ Sqlite database, schema, versions
			§ Connect - users might forget to close, etc
		○ Sqlite module overview
	• DataSet walkthrough
		○ Identity, also timestamps and state
		○ parameters
		○ Getting data
		○ Metadata feature


## BREAK [5min]

## I want MORE [5min]

What else does QCoDeS provide out of the box?

- q/a, other qcodes topics for self-exploration

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

- contrib-drivers
- Config
- DelegateAttributes
- Multiparameter
- Group parameter
- instrument functions
- Plotting support
- Instrument's functions
- Deprecation
- Linking datasets
- Extracting runs/datasets from db to another
- Monitor
- Interactive widget
- Legacy, loop-entangled, features
- writing drivers

## Presenter-only section

Some info, notes, and scratch-space for presenter.

qcodes logo means that the section(s) under it's level is(are) not ready.